In [2]:
%reload_ext autoreload
%autoreload 2

In [3]:
from typing import List, Optional, AsyncGenerator
from datetime import datetime
from pydantic import Field
from core import BaseCall, Msg
import instructor
from openai import AsyncOpenAI
import pandas as pd
from typing import Dict
from sampling import WeightedSampler, fetch_joined_data
from query_planner import QueryPlan
import enum
from typing import List
from pydantic import Field
from core import user_msg

from core import system_msg,user_msg,assistant_msg

from IPython.display import display,clear_output

In [4]:

class SearchQuery(BaseCall):
    query: Optional[List[str]] = Field(description="A specific, focused search query to find semantically relevant content.")
    include_speakers: Optional[List[str]] = Field(description="Speakers to include in search results May reference speaker groups")
    exclude_speakers: Optional[List[str]] = Field(description="Speakers to exclude from search results. May reference speaker groups")
    topic_type: Optional[List[str]] = Field(description="Types of topics to filter by")
    topic_name: Optional[List[str]] = Field(description="Topics name to include")

In [5]:
from sampling import fetch_joined_data,WeightedSampler

df = await fetch_joined_data()

In [6]:
sampler = WeightedSampler(df.drop(columns=['other_speakers']),date_column='meeting_timestamp')

Building vector index...


Batches:   0%|          | 0/100 [00:00<?, ?it/s]

In [43]:
from prompts import Prompts
prompts = Prompts()

In [44]:
prompts.perplexity

'You are Vexa, a helpful search assistant trained by Vexa AI. Your task is to deliver a concise and accurate response to a user\'s query, drawing from the given search results. Your answer must be precise, of high-quality, and written by an expert using an unbiased and journalistic tone. It is EXTREMELY IMPORTANT to directly answer the query. NEVER say "based on the search results" or start your answer with a heading or title. Get straight to the point. Your answer must be written in the same language as the query, even if language preference is different.\n\nYou MUST cite the most relevant search results that answer the query. Do not mention any irrelevant results. You MUST ADHERE to the following instructions for citing search results: - to cite a search result, enclose its index located above the summary with brackets at the end of the corresponding sentence, for example "Ice is less dense than water[1][2]." or "Paris is the capital of France[1][4][5]." - NO SPACE between the last w

In [7]:
samped_df = sampler.sample(n_samples=100)
topics = df['topic_name'].unique().tolist()
topic_types = df['topic_type'].unique().tolist()
speakers = df['speaker_name'].unique().tolist()

#SearchQuery = create_search_query_class(speakers, topic_types)

In [134]:
coworkers = [
    'Dmitriy Grankin',
    'Sergey Ryabenko',
    'Olga Nemirovskaya',
    'Alex Shevliakov',
    'Nick Frolov',
    'Jenish Mursidinov',
    'Dmitry Grankin',
    'Karine Stepanyan',
    'Dmitrii Bashkirov '
]

In [135]:
context = f"""
Available topics: {', '.join(topics)}
Topic types: {', '.join(topic_types)}
Speakers: {', '.join(speakers)}
Content: {' '.join(samped_df.to_csv())}
"""

In [164]:
knowledge = """
our product name is Vexa, meeting assistant, ai notetaker
exclude coworkers if asking about users
"""
speaker_groups = ['coworkers']

In [190]:
user_query = 'list all the products and companies that are meeting notetakers and assistants we know about'

In [191]:
messages = [
    system_msg(f'knwoledge: {knowledge},  speaker groups: {speaker_groups}, context {context}' ),
    user_msg(user_query)
            
            ]

In [192]:
async for plan in SearchQuery.call_stream(messages):
    clear_output()
    print(plan)


query=['meeting notetaker', 'meeting assistant', 'AI notetaker', 'transcription software'] include_speakers=None exclude_speakers=None topic_type=['product', 'company'] topic_name=None


In [193]:
plan

PartialSearchQuery(query=['meeting notetaker', 'meeting assistant', 'AI notetaker', 'transcription software'], include_speakers=None, exclude_speakers=None, topic_type=['product', 'company'], topic_name=None)

In [194]:

# Now sampler is an instance, not a class
# Assuming 'plan' is your SearchQuery instance
sampled_df = sampler.sample(
    query=plan.query,  # List of query strings
    exclude_speakers=coworkers if plan.exclude_speakers and 'coworkers' in plan.exclude_speakers else plan.exclude_speakers,
    include_speakers=plan.include_speakers,
    topic_type=plan.topic_type,
    topic_name=plan.topic_name,
    n_samples=1000,
    mode='combined',
    recency_weight=0.1,
    similarity_weight=0.1,
    filter_weight=1.
)

# Sort by combined score to see most relevant results first
sorted_results = sampled_df.sort_values('combined_score', ascending=False)

In [195]:
from instructor import patch, Partial
from openai import AsyncOpenAI

class Source(BaseCall):
    meeting_timestamp: datetime = Field(description="When this feedback was given")
    relevance: str = Field(description="Brief explanation of why this source is relevant to the query")

class SearchResponse(BaseCall):
    response: str = Field(description="A comprehensive analysis of the search results, focusing on key insights and patterns")
    sources: List[Source] = Field(description="List of relevant sources that support the response")
    


In [196]:
messages = [
    system_msg(f"""You are Vexa, a helpful search assistant trained by Vexa AI. Your task is to deliver a concise, accurate, and STRICTLY EVIDENCE-BASED response to the user's query based on internal meeting transcripts and notes.

CRITICAL RULES:
- NEVER make claims without direct evidence from meeting transcripts
- NEVER list items without specific citations for each
- NEVER create artificial balance or symmetry in responses
- NEVER generalize or summarize without direct quotes
- NEVER suggest the meeting notes are complete when they might be partial

COMPLETENESS AND LIMITATIONS:
- Always specify if the available meeting notes appear to be partial or limited
- Use phrases like "in our meeting transcripts we found..." or "during internal discussions, these competitors were mentioned..."
- Explicitly state when certain topics weren't discussed in the available meetings
- Acknowledge that discussions might have happened in meetings not captured here

EVIDENCE FORMAT:
- Use direct quotes from meetings whenever possible: "exact speaker words"[timestamp]
- Include meeting context: "During the product strategy meeting, Jane noted that..."[timestamp]
- Show specific numbers: "This competitor was discussed in 3 different meetings"[timestamp1][timestamp2][timestamp3]
- Include temporal context: "In the March 2024 planning meeting..."[timestamp]
- Explicitly state when topics weren't discussed: "No mentions of this competitor in available meeting transcripts"

FORBIDDEN PATTERNS:
- No balanced pro/con lists
- No generic feature descriptions
- No "people said..." without specific meeting quotes
- No symmetrical structuring of information
- No aggregating feedback without evidence from specific meetings
- No implying meeting coverage is complete

REQUIRED ELEMENTS:
- Raw, potentially messy presentation of meeting discussions
- Explicit gaps in meeting coverage
- Temporal context for each discussion
- Speaker and meeting context when relevant
- Direct quotes from meetings for subjective claims
- Clear indication of meeting transcript completeness

FILTERING RULES:
- Only consider content from allowed speakers. Exclude any content from: {plan.exclude_speakers}
{f'- Only include content from these speakers: {plan.include_speakers}' if plan.include_speakers else ''}

Meeting transcripts: {sorted_results[['summary', 'speaker_name', 'meeting_timestamp']].to_dict('records')}"""),
    user_msg(user_query)
]

In [197]:
from core import count_tokens
count_tokens(' '.join([m.content for m in messages]))


57324

In [198]:
user_query

'list all the products and companies that are meeting notetakers and assistants we know about'

In [199]:
async for partial_result in SearchResponse.call_stream(messages):
    clear_output()
    print(partial_result.response)  # Will show incremental updates to the model

In our meeting transcripts, the following products and companies related to meeting notetakers and assistants were mentioned:

1. **Otter** - "Otter is a transcription service that the speakers have subscribed to for their meetings." [2024-10-04 16:17:36.910000]

2. **Glitch One** - "Glitch One is a meeting recording tool mentioned in the context of capturing discussions." [2024-09-10 13:31:10.080000]

3. **Fireflies** - "Fireflies is a meeting assistant tool that records and transcribes meetings." [2024-09-10 09:28:46.960000]

4. **TLDW** - "TLDW is a transcription service that Pavel has experience with." [2024-08-31 18:21:02.335999]

5. **Nota** - "Nota is a meeting transcription tool that Ahmed found irritating due to its bot joining calls." [2024-08-30 10:29:05.146000]

6. **Vexa** - "Vexa is a meeting transcription tool that provides real-time transcripts and contextual support during conversations." [2024-08-30 10:29:05.146000]

7. **VEX.AI** - "VEX.AI is a product discussed in t

In [203]:
from IPython.display import Markdown
Markdown(partial_result.response)

In our meeting transcripts, the following products and companies related to meeting notetakers and assistants were mentioned:

1. **Otter** - "Otter is a transcription service that the speakers have subscribed to for their meetings." [2024-10-04 16:17:36.910000]

2. **Glitch One** - "Glitch One is a meeting recording tool mentioned in the context of capturing discussions." [2024-09-10 13:31:10.080000]

3. **Fireflies** - "Fireflies is a meeting assistant tool that records and transcribes meetings." [2024-09-10 09:28:46.960000]

4. **TLDW** - "TLDW is a transcription service that Pavel has experience with." [2024-08-31 18:21:02.335999]

5. **Nota** - "Nota is a meeting transcription tool that Ahmed found irritating due to its bot joining calls." [2024-08-30 10:29:05.146000]

6. **Vexa** - "Vexa is a meeting transcription tool that provides real-time transcripts and contextual support during conversations." [2024-08-30 10:29:05.146000]

7. **VEX.AI** - "VEX.AI is a product discussed in the meeting, aimed at enhancing user experience during meetings." [2024-09-16 15:35:48.424000]

8. **ChatGPT** - "ChatGPT is an AI language model developed by OpenAI, used for generating human-like text." [2024-09-16 11:30:53.364000]

9. **Claude** - "Claude is another AI tool mentioned alongside ChatGPT in the context of user research." [2024-09-17 09:49:06.520000]

10. **Ask.ai** - "ask.ai is a tool mentioned for generating code and assisting in development tasks." [2024-09-23 11:09:16.803999] 

These products and companies were discussed in various contexts related to meeting transcription and assistance. Note that the list may not be exhaustive as discussions might have occurred in meetings not captured here.

In [204]:
# Get timestamps from partial_result.sources
source_timestamps = [source.meeting_timestamp for source in partial_result.sources]

filtered_results = sorted_results[sorted_results['meeting_timestamp'].isin(source_timestamps)]
filtered_results.sort_values('meeting_timestamp',ascending=True)

,summary_index,summary,details,referenced_text,topic_name,topic_type,meeting_id,meeting_timestamp,speaker_name,filter_score,recency_score,similarity_score,combined_score
1622,17,Zoom is another online meeting application tha...,Dmitry states that they do not use Zoom at all...,,Zoom,application,247a9f4e-1182-401e-bf61-95a6c3a12137,2024-08-30 10:29:05.146000,Dmitry Grankin,1.0,0.000018,0.386994,0.000315
1633,20,The dashboard is a feature of Vexa that will b...,Dmitry mentions upcoming improvements to the d...,,dashboard,concept,247a9f4e-1182-401e-bf61-95a6c3a12137,2024-08-30 10:29:05.146000,Dmitry Grankin,1.0,0.000018,0.255827,0.000311
1656,5,Ahmed Abdelaziz emphasized the importance of t...,He suggested that Vexa should focus on definin...,"Ahmed Abdelaziz: I think, I think, yeah, this...",Discussion on marketing strategies and outreach,strategy,247a9f4e-1182-401e-bf61-95a6c3a12137,2024-08-30 10:29:05.146000,Ahmed Abdelaziz,1.0,0.000018,0.261133,0.000312
1657,6,Ahmed Abdelaziz explained the credit-based pri...,"The starter package costs $100 for 2,500 credi...",Ahmed Abdelaziz: it's we have it's running by...,Details on credit-based pricing for Forager.ai...,pricing,247a9f4e-1182-401e-bf61-95a6c3a12137,2024-08-30 10:29:05.146000,Ahmed Abdelaziz,1.0,0.000018,0.288107,0.000312
1646,22,Email is a key data point provided by Forager....,Ahmed emphasizes the importance of having accu...,,email,concept,247a9f4e-1182-401e-bf61-95a6c3a12137,2024-08-30 10:29:05.146000,Ahmed Abdelaziz,1.0,0.000018,0.388749,0.000316
...,...,...,...,...,...,...,...,...,...,...,...,...,...
185,13,Next.js is a React framework for building serv...,Sergey mentions Next.js as a component library...,Sergey Ryabenko: you need to have installed a...,Next.js,product,2991351f-26e5-4d49-b1a0-daeaa77b18e5,2024-09-23 11:09:16.803999,Sergey Ryabenko,1.0,0.000195,0.222918,0.000310
873,4,The team decided to prioritize personal messag...,Olga noted that personal messages yield better...,Olga Nemirovskaya: Надо было с самого начала ...,Personal messaging vs. public posts,decision,fb2aea72-f75d-4790-99dc-bbc4c7219a31,2024-10-04 16:17:36.910000,Olga Nemirovskaya,1.0,0.000587,0.260600,0.000312
872,3,The team discussed the effectiveness of email ...,Olga expressed concerns that email outreach ma...,"Olga Nemirovskaya: Ну, мне кажется, это будет...",Email outreach effectiveness,concern,fb2aea72-f75d-4790-99dc-bbc4c7219a31,2024-10-04 16:17:36.910000,Olga Nemirovskaya,1.0,0.000587,0.309240,0.000313
875,21,"Compliance is a key topic in the meeting, part...",The speakers discuss how to ensure their outre...,Olga Nemirovskaya: В комплайент.,compliance,concept,fb2aea72-f75d-4790-99dc-bbc4c7219a31,2024-10-04 16:17:36.910000,Dmitriy Grankin,1.0,0.000587,0.396140,0.000316


In [205]:
sorted_results.sort_values('meeting_timestamp',ascending=True)

,summary_index,summary,details,referenced_text,topic_name,topic_type,meeting_id,meeting_timestamp,speaker_name,filter_score,recency_score,similarity_score,combined_score
1290,13,Кейнборды are discussed as part of the marketi...,Olga and Dmitriy discuss the status of Кейнбор...,"Olga Nemirovskaya: кончился, мы с ним обсужда...",Кейнборды,topic,cd43879a-0b16-4473-9744-04ae567626ff,2024-08-26 19:14:18.976,Olga Nemirovskaya,1.0,0.000012,0.175495,0.000309
1278,2,The team is in discussions with a TikTok influ...,Olga mentioned that there has been a delay in ...,Olga Nemirovskaya: По ТикТоку что-то там дево...,Discussion on TikTok collaboration,task,cd43879a-0b16-4473-9744-04ae567626ff,2024-08-26 19:14:18.976,Olga Nemirovskaya,1.0,0.000012,0.418606,0.000316
1305,28,Action points are tasks or follow-ups discusse...,Olga expresses dissatisfaction with the action...,Olga Nemirovskaya: еще кстати вот у саши мне ...,action points,concept,cd43879a-0b16-4473-9744-04ae567626ff,2024-08-26 19:14:18.976,Olga Nemirovskaya,1.0,0.000012,0.319986,0.000313
1309,32,Анализ refers to the analysis of user data and...,The meeting emphasizes the importance of analy...,"Olga Nemirovskaya: Да, это будет удобнее.",анализ,concept,cd43879a-0b16-4473-9744-04ae567626ff,2024-08-26 19:14:18.976,Dmitriy Grankin,1.0,0.000012,0.270303,0.000312
1284,7,There are concerns about the clarity and relev...,Olga expressed dissatisfaction with the action...,Olga Nemirovskaya: еще кстати вот у саши мне ...,Action points and task management,concern,cd43879a-0b16-4473-9744-04ae567626ff,2024-08-26 19:14:18.976,Olga Nemirovskaya,1.0,0.000012,0.438886,0.000317
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1236,8,Lisbon is the city where Dmitry Grankin curren...,Dmitry mentions living in Lisbon and having ch...,Dmitry Grankin: uh lisbon i live in lisbon i ...,Lisbon,location,50d5d659-3a57-42f2-8f32-31c7145ebe91,2024-10-24 09:32:32.980,Dmitry Grankin,1.0,0.003923,0.173798,0.000309
1240,12,Zoom is another video conferencing tool mentio...,"Similar to Google Meet, Zoom is referenced as ...",Dmitry Grankin: yes definitely you do and uh ...,Zoom,product,50d5d659-3a57-42f2-8f32-31c7145ebe91,2024-10-24 09:32:32.980,Dmitry Grankin,1.0,0.003923,0.474893,0.000318
1242,14,A Chrome Extension developed by Dmitry to enha...,Dmitry discusses the development of a Chrome e...,"Özay Demirezen: Okay, yeah, this,",Chrome Extension,product,50d5d659-3a57-42f2-8f32-31c7145ebe91,2024-10-24 09:32:32.980,Dmitry Grankin,1.0,0.003923,0.413146,0.000316
1237,9,Lidl is a supermarket chain mentioned by Dmitr...,"Dmitry confirms he is still in Lisbon, implyin...",Dmitry Grankin: uh lisbon i live in lisbon i ...,Lidl,company,50d5d659-3a57-42f2-8f32-31c7145ebe91,2024-10-24 09:32:32.980,Dmitry Grankin,1.0,0.003923,0.243771,0.000311
